<a href="https://colab.research.google.com/github/thinnarat007/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

#home

In [19]:
import pandas as pd
import numpy as np



In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:

import os
os.chdir('/content/drive/MyDrive/bsc_dpdm24_data_thin/midterm')


In [162]:
!pip install openpyxl
import pandas as pd
import numpy as np
# Instead of pd.read_xlsx, use pd.read_excel and specify the engine
data_house = pd.read_excel("/content/drive/MyDrive/bsc_dpdm24_data_thin/midterm/Data House.xlsx", engine='openpyxl')

In [67]:
data_house.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# prompt: data_house มีคลอลัมอะไรบ้าง

data_house.columns

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden'],
      dtype='object')

In [163]:
# เลือกเฉพาะคอลัมน์ที่ต้องการ
columns_needed = [
    'renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va',
    'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
    'direction', 'furniture', 'usable_area', 'number_of_parking',
    'sell_price', 'district', 'sub_district', 'zone'
]
data_filtered = data_house[columns_needed]


In [72]:
data_filtered.shape

(1656, 15)

In [73]:
data_filtered.head()

,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,no,0,1,45.0,2.0,2.0,2,ไม่ระบุ,ไม่มี,244.0,2.0,390000.0,ชนบท,ศรีบุญเรือง,NaN
1,no,0,0,88.0,1.0,1.0,1,ไม่ระบุ,ไม่มี,164.0,3.0,820000.0,น้ำพอง,ทรายมูล,NaN
2,no,1,0,8.0,1.0,11.0,11,ไม่ระบุ,ไม่ระบุ,489.0,9.0,3260000.0,มัญจาคีรี,กุดเค้า,NaN
3,no,0,0,61.6,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,203.0,2.0,5000000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,no,0,0,72.3,2.0,3.0,2,ไม่ระบุ,ไม่ระบุ,164.0,2.0,3500000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน


In [164]:
# prompt: ตรวจสอบ missing ของ data_filtered

# Check for missing values in the DataFrame
missing_values = data_filtered.isnull().sum()
missing_values

,0
renovate,211
number_of_rai,0
number_of_ngan,0
number_of_va,0
number_of_story,549
number_of_bedroom,27
number_of_bathroom,0
direction,0
furniture,0
usable_area,920


In [165]:
data_filtered['renovate'] = data_filtered['renovate'].fillna(data_filtered['renovate'].mode()[0])


<ipython-input-165-7a238b10aea0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['renovate'] = data_filtered['renovate'].fillna(data_filtered['renovate'].mode()[0])


In [166]:

# คำนวณค่าเฉลี่ยของ 'number_of_story' และเติมค่าหายไปด้วยค่าเฉลี่ย (ปัดขึ้นเป็นจำนวนเต็ม)
mean_story = np.mean(data_filtered['number_of_story'].dropna())  # คำนวณค่าเฉลี่ย (ไม่รวมค่า NaN)
data_filtered['number_of_story'] = data_filtered['number_of_story'].fillna(np.ceil(mean_story).astype(int))

<ipython-input-166-9810446c0e8f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['number_of_story'] = data_filtered['number_of_story'].fillna(np.ceil(mean_story).astype(int))


In [167]:
# คำนวณค่าเฉลี่ยของ 'number_of_bedroom' และเติมค่าหายไปด้วยค่าเฉลี่ย (ปัดขึ้นเป็นจำนวนเต็ม)
mean_bedroom = np.mean(data_filtered['number_of_bedroom'].dropna())  # คำนวณค่าเฉลี่ย (ไม่รวมค่า NaN)
data_filtered['number_of_bedroom'] = data_filtered['number_of_bedroom'].fillna(np.ceil(mean_bedroom).astype(int))


<ipython-input-167-389567b11298>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['number_of_bedroom'] = data_filtered['number_of_bedroom'].fillna(np.ceil(mean_bedroom).astype(int))


In [168]:
# คำนวณค่ามัธยฐานของ 'usable_area' และเติมค่าหายไปด้วยค่ามัธยฐาน
median_usable_area = np.median(data_filtered['usable_area'].dropna())  # คำนวณค่ามัธยฐาน (ไม่รวมค่า NaN)
data_filtered['usable_area'] = data_filtered['usable_area'].fillna(median_usable_area)


<ipython-input-168-abf845fe787a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['usable_area'] = data_filtered['usable_area'].fillna(median_usable_area)


In [169]:
# คำนวณค่ามัธยฐานของ 'number_of_parking' และเติมค่าหายไปด้วยค่ามัธยฐาน
median_parking = np.median(data_filtered['number_of_parking'].dropna())  # คำนวณค่ามัธยฐาน (ไม่รวมค่า NaN)
data_filtered['number_of_parking'] = data_filtered['number_of_parking'].fillna(median_parking)


<ipython-input-169-fe3fc5475222>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['number_of_parking'] = data_filtered['number_of_parking'].fillna(median_parking)


In [170]:
# ลบแถวที่มีค่า missing ใน 'sell_price'
data_filtered = data_filtered.dropna(subset=['sell_price'])

In [171]:
# คำนวณค่าพบบ่อยที่สุด (mode) และเติมค่าหายไปใน 'sub_district'
mode_sub_district = data_filtered['sub_district'].mode()[0]  # ค่า mode ของ 'sub_district'
data_filtered['sub_district'] = data_filtered['sub_district'].fillna(mode_sub_district)


In [172]:
# เติมค่า 'unknown' ในคอลัมน์ 'zone' ที่มีค่า missing
data_filtered['zone'] = data_filtered['zone'].fillna('unknown')


In [173]:
import pandas as pd

# กำหนดฟังก์ชันตรวจสอบ outliers โดยใช้ IQR
def detect_outliers_IQR(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# คอลัมน์ที่ต้องการตรวจสอบ outliers
columns_to_check = ['number_of_rai', 'number_of_ngan', 'number_of_va', 'number_of_story',
                    'number_of_bedroom', 'number_of_bathroom', 'usable_area', 'number_of_parking', 'sell_price']

# ตรวจสอบ outliers สำหรับแต่ละคอลัมน์
outlier_results = {}
for col in columns_to_check:
    outlier_results[col] = detect_outliers_IQR(data_filtered, col)

# แสดงผลลัพธ์ outliers สำหรับแต่ละคอลัมน์
for col, outliers in outlier_results.items():
    print(f"Outliers in column {col}:")
    print(outliers)
    print("\n")


Outliers in column number_of_rai:
     renovate  number_of_rai  number_of_ngan  number_of_va  number_of_story  \
2          no              1               0           8.0              1.0   
20         no              4               1          66.0              2.0   
25         no              1               0           0.0              2.0   
34         no              3               2          92.0              2.0   
35         no              7               3          39.8              2.0   
...       ...            ...             ...           ...              ...   
1547       no             25               0           0.0              2.0   
1566       no              1               1         101.0              2.0   
1574       no              7               0          43.0              2.0   
1623       no              1               1           1.0              2.0   
1643       no              1              95         780.0              2.0   

      number_of_b

In [174]:
data_filtered = pd.get_dummies(data_filtered, columns=['renovate'], drop_first=True)


In [175]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_filtered[['number_of_rai', 'number_of_ngan', 'number_of_va']] = scaler.fit_transform(data_filtered[['number_of_rai', 'number_of_ngan', 'number_of_va']])


In [176]:
data_filtered = pd.get_dummies(data_filtered, columns=['number_of_bedroom'], drop_first=True)


In [177]:
data_filtered = pd.get_dummies(data_filtered, columns=['direction'], drop_first=True)


In [178]:
scaler = MinMaxScaler()
data_filtered['usable_area'] = scaler.fit_transform(data_filtered[['usable_area']])


In [179]:
data_filtered = pd.get_dummies(data_filtered, columns=['district'], drop_first=True)


In [180]:
data_filtered = pd.get_dummies(data_filtered, columns=['sub_district'], drop_first=True)


In [181]:
data_filtered = pd.get_dummies(data_filtered, columns=['zone'], drop_first=True)


In [182]:
# ใช้ One-Hot Encoding สำหรับคอลัมน์ 'furniture'
data_filtered = pd.get_dummies(data_filtered, columns=['furniture'], drop_first=True)


#7-11 and schoo

In [32]:
import json

In [33]:
# อ่านไฟล์ 7-eleven.json
with open('/content/drive/MyDrive/bsc_dpdm24_data_thin/midterm/7-eleven.json', 'r') as f:
    seven_eleven = json.load(f)

# อ่านไฟล์ school.json
with open('/content/drive/MyDrive/bsc_dpdm24_data_thin/midterm/school.json', 'r') as f:
    schools = json.load(f)

In [37]:
# ตรวจสอบว่า seven_eleven และ schools เป็น dict
print("7-Eleven keys:", list(seven_eleven.keys())[:5])  # แสดง keys ตัวอย่าง
print("School keys:", list(schools.keys())[:5])        # แสดง keys ตัวอย่าง

7-Eleven keys: ['1', '4', '6', '7', '9']
School keys: ['1', '4', '6', '7', '9']


In [93]:
from geopy.distance import geodesic

def calculate_distance(lat1, lon1, lat2, lon2):
    """คำนวณระยะทางระหว่างสองจุดที่มีพิกัด latitude และ longitude"""
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)
    return geodesic(point1, point2).kilometers

# ตัวอย่างการคำนวณระยะทาง
lat_house, lon_house = 16.402, 102.712  # พิกัดบ้าน
lat_7eleven, lon_7eleven = 16.403, 102.713  # พิกัด 7-11

distance = calculate_distance(lat_house, lon_house, lat_7eleven, lon_7eleven)
print(f"Distance: {distance} km")

Distance: 0.15380587401261694 km


In [95]:
distance_7_11 = 0.15380587401261694  # ค่าระยะทางที่คำนวณได้จาก 7-11
distance_school = 0.15380587401261694  # ค่าระยะทางที่คำนวณได้จากโรงเรียน

# ตรวจสอบว่าระยะทางน้อยกว่า 1 กิโลเมตรหรือไม่
d1000_7_11 = 1 if distance_7_11 < 1 else 0  # ถ้าน้อยกว่า 1 กิโลเมตรให้เป็น 1
d1000_school = 1 if distance_school < 1 else 0  # ถ้าน้อยกว่า 1 กิโลเมตรให้เป็น 1

# เพิ่มค่าคอลัมน์ d1000-7/11 และ d1000-school ใน DataFrame
data_filtered['d1000-7/11'] = d1000_7_11
data_filtered['d1000-school'] = d1000_school

<ipython-input-95-6660f0dd223f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['d1000-7/11'] = d1000_7_11
<ipython-input-95-6660f0dd223f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['d1000-school'] = d1000_school


In [38]:
data_house.shape

(1656, 47)

In [107]:

# ตรวจสอบข้อมูลที่เพิ่มใหม่
print(data_filtered[['d1000-7/11', 'd1000-school']].head())

   d1000-7/11  d1000-school
0           1             1
1           1             1
2           1             1
3           1             1
4           1             1


In [121]:
data_filtered.shape

(1655, 217)

,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bathroom,usable_area,number_of_parking,sell_price,d1000-7/11,d1000-school,...,zone_สามเหลี่ยม-กังสดาล-ม.ขอนแก่น,zone_เหล่านาดี-บ้านสะอาด-บ้านโจด,zone_แดงใหญ่,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,furniture_มีบางส่วน,furniture_ไม่มี,furniture_ไม่ระบุ
0,0.000000,0.010526,0.057252,2.0,2,0.005989,2.0,0.003671,1,1,...,False,False,False,False,False,False,False,False,True,False
1,0.000000,0.000000,0.111959,1.0,1,0.004018,3.0,0.007771,1,1,...,False,False,False,False,False,False,False,False,True,False
2,0.014925,0.000000,0.010178,1.0,11,0.012028,9.0,0.031036,1,1,...,False,False,False,False,False,False,False,False,False,True
3,0.000000,0.000000,0.078372,2.0,3,0.004979,2.0,0.047626,1,1,...,False,False,False,False,False,False,False,False,False,True
4,0.000000,0.000000,0.091985,2.0,2,0.004018,2.0,0.033324,1,1,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,0.000000,0.000000,0.092875,1.0,1,0.002656,0.0,0.004123,1,1,...,False,False,False,False,False,False,False,False,False,True
1652,0.000000,0.000000,0.106870,1.0,1,0.002908,0.0,0.006766,1,1,...,False,False,False,False,False,False,False,False,False,True
1653,0.000000,0.010526,0.057252,2.0,1,0.004622,0.0,0.004253,1,1,...,False,False,False,False,False,False,False,False,False,True
1654,0.000000,0.010526,0.075064,2.0,1,0.005078,0.0,0.008629,1,1,...,False,False,False,False,False,False,False,False,False,True


In [183]:
# แยก Features และ Target
X = data_filtered.drop(columns=['sell_price'])
y = data_filtered['sell_price']


In [184]:
from sklearn.model_selection import train_test_split

# แบ่งข้อมูลเป็น Training และ Testing Set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [192]:
import lightgbm as lgb

# สร้างโมเดล LightGBM
model_lgb = lgb.LGBMRegressor(n_estimators=100, max_depth=3)

# ฝึกโมเดล
model_lgb.fit(X_train, y_train)

# ทำนายผล
y_pred_lgb = model_lgb.predict(X_test)

# คำนวณ MSE และ R^2
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
r2_lgb = r2_score(y_test, y_pred_lgb)

print(f"Mean Squared Error (LightGBM): {mse_lgb}")
print(f"R^2 Score (LightGBM): {r2_lgb}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000442 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 473
[LightGBM] [Info] Number of data points in the train set: 1324, number of used features: 48
[LightGBM] [Info] Start training from score 3613456.984139
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [195]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    'n_estimators': [100, 200],  # ลดจำนวนค่าที่จะทดสอบ
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}



rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_


Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}


In [196]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# สร้างโมเดลใหม่
rf_best = RandomForestRegressor(
    max_depth=None,
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=200
)

# ทำนายผลด้วยโมเดลที่ดีที่สุด
y_pred = grid_search.best_estimator_.predict(X_test)

# คำนวณ MSE และ R^2
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)



Mean Squared Error: 7963544459609.52
R^2 Score: 0.42808252469269437


In [201]:
# ตรวจสอบชื่อคอลัมน์ใน evaluations
print(evaluations.columns)


Index(['ID ผู้ประเมิน', 'post id', 'ID', 'property_type',
       'ความมั่นใจในการเมินราคาทรัพย์', 'ราคาทรัพย์', 'ประเมินราคาซื้อ-ขาย',
       'ปัจจัยในการประเมินราคาซื้อ-ขาย', 'เหตุผลไม่มั่นใจราคาประเมิน',
       'ให้คะแนนทรัพย์'],
      dtype='object')
